In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import model_selection, tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC

In [ ]:
stroke = pd.read_csv('../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv',index_col=[0])
stroke

<br>
<ins>ตัวอย่างคอลัมน์</ins>

| Column | Description |
| :--------- | :---------- |
| id	| ลำดับ |
| gender	| เพศ |
| age	| อายุ |
| hypertension	| เป็นโรคหลอดโรคความดันโลหิตสูงหรือไม่? |
| heart_disease	| เป็นโรคหัวใจหรือไม่? |
| ever_married | สถานะการแต่งงาน |
| work_type	| อาชีพ|
| Residence_type	| ที่อยู่อาศัย|
| avg_glucose_level	| ค่าเฉลี่ยน้ำตาลในเลือด|
| bmi | ดัชนีมวลกาย |
| smoking_status | การสูบบุหรี่ |
| stroke | เป็นโรคหลอดเลือดสมองหรือไม่? |


<br>

อธิบายข้อมูลเพิ่มเติม
<br>1) id: เลขแทนข้อมูลแต่ละแถวที่แตกต่างกัน
<br>2) gender: "Male"(เพศชาย), "Female"(เพศหญิง), "Other"(เพศอื่นๆ)
<br>3) age: อายุ
<br>4) hypertension: 0 ถ้าผู้ป่วยไม่เป็นโรคความดันโลหิตสูง, 1 ถ้าผู้ป่วยเป็นโรคความดันโลหิตสูง
<br>5) heart_disease: 0 ถ้าผู้ป่วยไม่เป็นโรคหัวใจ, 1 ถ้าผู้ป่วยเป็นโรคหัวใจ
<br>6) ever_married: เคยแต่งงานมาก่อนหรือไม่ -> "No"(ไม่เคยสมรส), "Yes"(เคยสมรส)
<br>7) work_type: "children"(เด็ก), "Govt_jov"(งานรัฐบาล), "Never_worked"(ไม่ทำงาน), "Private"(ธุรกิจส่วนตัว),"Self-employed"(ประกอบอาชีพอิสระ)
<br>8) Residence_type: "Rural"(อาศัยอยู่นอกเมือง), "Urban"(อาศัยอยู่ในเมือง)
<br>9) avg_glucose_level: ค่าเฉลี่ยน้ำตาลในเลือดทศนิยมสูงสุดสองตำแหน่ง
<br>10) bmi: body mass index(ดัชนีมวลกาย)
<br>11) smoking_status: "formerly smoked"(เคยสูบบุหรี่ก่อนหน้านี้), "never smoked"(ไม่เคยสูบบุหรี่), "smokes"(สูบบุหรี่), "Unknown"(ไม่ทราบข้อมูล)
<br>12) stroke: 1 ถ้าผู้ป่วยเป็นโรคหลอดเลือดสมอง, 0 ถ้าผู้ป่วยไม่เป็นโรคหลอดเลือดสมอง

In [ ]:
stroke = stroke[(~stroke.bmi.isna()) & (~stroke.avg_glucose_level.isna()) & (stroke.smoking_status != 'Unknown')]

In [ ]:
stroke = stroke.drop(['ever_married', 'work_type', 'Residence_type'], axis=1)

In [ ]:
stroke

`y` คือคำตอบของชุดข้อมูล

In [ ]:
y = stoke.stroke

`X` คือชุดตัวแปรต้นของข้อมูล

โดยทำการเอาผลเฉลยออก เพื่อเตรียมการทำ Model

In [ ]:
X = stroke.drop("stroke", axis=1)
X = pd.get_dummies(X)

ทำการแบ่งกลุ่มของข้อมูล

เพื่อที่จะทำการ Train model และ test model ด้วยชุดข้อมูลที่ model ไม่เคยเห็นมาก่อน
* ข้อมูลชุด Train model        `X_train`
* ชุดข้อมูลสำหรับ test model    `X_test`
* ชุดคำตอบของชุด Train model  `y_train`
* ชุดคำตอบของ test model     `y_test`

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y)

## K-Neighbor Classifier

Algorithms K-Neighbor 

หา `n_neighbors` ที่ดีที่สุดด้วย `GridSearchCV`
โดย `n_neighbors` ที่ทำการ brute force คือ 1 3 5 7 9 11 13 15 17 19 21

In [ ]:
params = {'n_neighbors': [ i for i in range(1, 22, 2)]}
params

In [ ]:
test_knn = KNeighborsClassifier()
gs_cv = GridSearchCV(test_knn, params)
gs_cv.fit(X_train, y_train)
best_k = gs_cv.best_params_['n_neighbors']
best_k

ค่า K ที่ทำให้จัดกลุ่มแล้วได้ผลลัพธ์ที่แม่นยำที่สุดคือ 15

In [ ]:
kNN = KNeighborsClassifier(best_k)
kNN.fit(X_train, y_train)

In [ ]:
kNN.score(X_test, y_test)

### ความแม่นยำของ Model ที่ใช้ Algorithm K-Neighbor Classifier คือ 95.45%

## K-Neighbor ร่วมกับการแบ่งกลุ่มข้อมูลแบบ KFold
KFold เป็นการ suffle ข้อมูลร่วมกับการแบ่งข้อมูล 

ทำให้ Dataset ที่มีการ Bias ของข้อมูล Train Model ได้แม่นยำมากขึ้น

In [ ]:
y_train_kfold = y_test_kfold = pd.Series([])
kf = KFold(n_splits=2)

In [ ]:
kNN = KNeighborsClassifier(best_k)

In [ ]:
for train_index, test_index in kf.split(X):
    X_train_kfold = X.iloc[train_index]
    X_test_kfold = X.iloc[test_index]
    y_train_kfold = (y.iloc[train_index])
    y_test_kfold = (y.iloc[test_index])
    kNN.fit(X_train_kfold, y_train_kfold)

In [ ]:
kNN.score(X_test_kfold, y_test_kfold)

### เมื่อใช้ Algorithm K-neighbor ร่วมกับวิธีการแบ่งกลุ่มข้อมูลด้วย KFold ทำให้ Model มีความแม่นยำถึง 98.66% 

## Decision Tree
   Decision Tree เป็น Algorithms การตัดสินใจ โดยจะแยกข้อมูลการตัดสินใจไปเรื่อยๆจนกว่าจะไปถึง condition สุดท้าย
   
   input ที่เข้ามาจะเริ่มต้นจาก Root Node ของต้นไม้ แล้วจะถูกตัดสินใจๆไปเรื่อยๆจนถึง Leaf Node

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train,y_train)

In [ ]:
dt.score(X_test, y_test)

### ความแม่นยำของ Model ที่ใช้ Algorithm Decision Tree Classifier คือ 89.15%

### แผนภาพของ Model Decision Tree Classifier

In [ ]:
import graphviz 
dot_data = tree.export_graphviz(dt, out_file=None) 
graph = graphviz.Source(dot_data) 
graph